In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cmapPy

In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, precision_score, label_ranking_average_precision_score
from sklearn.model_selection import GridSearchCV
from sklearn import svm, preprocessing
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import random_split, Dataset,DataLoader

In [ ]:
import pickle
with open('data/reactome_term_dicts.pickle', 'rb') as handle:
    reactome_dict = pickle.load(handle)

## DepMap response data to various drugs
GI50 (proliferation rate change) per cell line for various drugs

Formatting the data to be cell_line by drug name. Data is stored in multiple files using various namings conventions. Converting to CCLE and generic drug name

In [ ]:
cl_response = pd.read_csv('data/depmap_data/primary-screen-replicate-collapsed-logfold-change.csv')

cl_response.rename({'Unnamed: 0': 'cell_line'}, axis=1, inplace=True)
cell_lines_in_treatment = set(cl_response.cell_line.unique())

In [ ]:
len(cell_lines_in_treatment)

In [ ]:
cl_response.set_index('cell_line', inplace=True)

cl_response.head(5)

In [ ]:
cl_response.shape

In [ ]:
# canonical cell line names
cell_lines = pd.read_csv('data/depmap_data/primary-screen-cell-line-info.csv')
# quick dict for naming
rowname_to_cell_line = cell_lines[['row_name', 'ccle_name']].copy()
rowname_to_cell_line.set_index('row_name', inplace=True)
rowname_to_cell_line

In [ ]:
# rename cell lines
cl_response.rename(rowname_to_cell_line.to_dict()['ccle_name'], inplace=True)
cl_response = cl_response.reset_index().set_index('cell_line')
# cl_response.dropna(inplace=True, axis=0)
cl_response.fillna(value=cl_response.mean(), inplace=True)
cl_response.sort_index(inplace=True, ascending=False)
cl_response.head(5)

In [ ]:
cl_response

In [ ]:
cl_response.shape

In [ ]:
# get canonical drug names, 
drugs = pd.read_csv('data/depmap_data/primary-screen-replicate-collapsed-treatment-info.csv')
drugs.head()

In [ ]:
rename_drug = drugs[['column_name', 'name']].drop_duplicates().set_index('column_name').to_dict()['name']
#renmae column names to drug names
cl_response.rename(columns=rename_drug, inplace=True)
cl_response.head(5)

In [ ]:
plt.figure(figsize=(12,12))
sclc = cl_response.loc[cl_response.index.str.contains('LUNG')]
plt.title(f'{sclc.shape[1]} drugs: {sclc.shape[0]} cell lines')

sns.heatmap(sclc)
plt.savefig('heatmap_depmap_gi50.png')

## RNAseq from CCLE


In [ ]:
from cmapPy.pandasGEXpress.parse import parse
# ccle_rna = parse('data/ccle/CCLE_RNAseq_genes_rpkm_20180929.gct')
ccle_rna = pd.read_csv('data/depmap_data/CCLE_expression.csv')
ccle_rna.head()

In [ ]:
rename_gene = ccle_rna.row_metadata_df.reset_index()[['rid', 'Description']].drop_duplicates().set_index('rid').to_dict()['Description']

In [ ]:
ccle_rna_d = ccle_rna.data_df.copy()
ccle_cl_names = list(ccle_rna_d.columns)

ccle_rna_d.reset_index(inplace=True)

ccle_rna_d.columns.name = None
ccle_rna_d.head(5)

In [ ]:
# rename ENSG ids to HGNC names
ccle_rna_d['rid'] = ccle_rna_d['rid'].map(rename_gene)
ccle_rna_d.head()

In [ ]:
# some genes have duplicate measurments. Weird for RNAseq but will average out for meantime
ccle_rna_d = ccle_rna_d.groupby(['rid']).mean().reset_index()
ccle_genes = list(ccle_rna_d['rid'])
ccle_genes[:10]

In [ ]:
ccle_rna_d.set_index('rid', inplace=True)
ccle_rna_d.head(5)

In [ ]:
# Pivot so cell line is each row
ccle_rna_d = ccle_rna_d.T

In [ ]:
ccle_rna_d.shape

In [ ]:
ccle_rna_d.index.name =  'cell_line'
ccle_rna_d.columns.name = None
ccle_rna_d.head(5)

In [ ]:
ccle_rna_d.std()

In [ ]:
good_std = list(ccle_rna_d.std()[ccle_rna_d.std() > 5].index.values)
print(len(good_std))

In [ ]:
cl_response.head(5)
drugs = list(cl_response.columns)
drugs[:5]

In [ ]:
cl_response.shape

In [ ]:
cl_response.head()

In [ ]:
# proteomics = pd.read_csv('data/ccle/protein_quant_current_normalized.csv.gz')
# proteomics.head()
# cl_col = [i for i in sorted(proteomics.columns.values) if 'Ten' in i]
# cl_prefix = [i.split('_Ten')[0] for i in cl_col]

# print(set(cl_prefix).intersection(set(ccle_cl_names)))

In [ ]:
reactome_df = pd.DataFrame.from_dict(reactome_dict)
reactome_df.head()

In [ ]:
# weird case where some of the drugs are genes? Removing from drug side
# all_but = set(ccle_rna_d.columns).intersection(cl_response.columns)

all_but = set(cl_response.columns)
gene_drugs = ['LTA', 'NPPB', 'PRIMA1']
for i in gene_drugs:
    all_but.discard(i)
# [genes expression cols] + [drug response]
merged = ccle_rna_d.join(cl_response[all_but], how='inner')
# average cell line expression for drugs/expression
# should probably do this before and see where its coming from
merged = merged.reset_index().groupby(['cell_line']).mean()
merged = merged.loc[:,~merged.columns.duplicated()]
merged.dropna(inplace=True)
print(merged.shape)

In [ ]:
merged.to_hdf('data.h5', key='df', mode='w')
merged = pd.read_hdf('data.h5', 'df')

In [ ]:
from sklearn.linear_model import ElasticNet



def create_elasticnet(df, drugname, subset_list, save_name):
    subset_genes = list(sorted(set(subset_list).intersection(set(df.columns.values))))
    X = df[subset_genes].copy()
    y = df[drugname].copy()
    X = preprocessing.StandardScaler().fit(X).transform(X)
#     poly = PolynomialFeatures(degree=2)
#     X = poly.fit_transform(X)
     
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2, 
        shuffle=True, 
        random_state=69,
    )
    
    print(X_train.shape, y_train.shape)
#     model = svm.SVR(gamma=0.001, C=100.)
#     model.fit(X_train, y_train)
    model = ElasticNet(random_state=0)
    model.fit(X_train, y_train)

    plt.subplot(111)
    t_preds = model.predict(X_train)
    sns.regplot(y_train, t_preds, label='training')
    
    preds = model.predict(X_test)
    error = mean_squared_error(y_test, preds)
    sns.regplot(y_test, preds, label='prediction')
    plt.legend()
    plt.suptitle(f"{drugname} {save_name} : RMSE = {error}")
    
create_elasticnet(merged, 'ABT-737', apop_genes, 'apoptosis_subset')

In [ ]:
apop_genes = reactome_df.loc[reactome_df.term=='apoptosis homo sapiens r-hsa-109581']['gene_list'].values
apop_genes = list(apop_genes[0])    

In [ ]:

def create_svm(df, drugname, subset_list, save_name):
    subset_genes = list(sorted(set(subset_list).intersection(set(df.columns.values))))
    X = df[subset_genes].copy()
    y = df[drugname].copy()
    X = preprocessing.StandardScaler().fit(X).transform(X)
#     poly = PolynomialFeatures(degree=2)
#     X = poly.fit_transform(X)
     
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2, 
        shuffle=True, 
        random_state=69,
    )
    
    print(X_train.shape, y_train.shape)
    model = svm.SVR(gamma=0.001, C=100.)
    model.fit(X_train, y_train)
    

    plt.subplot(111)
    t_preds = model.predict(X_train)
    sns.regplot(y_train, t_preds, label='training')
    
    preds = model.predict(X_test)
    error = mean_squared_error(y_test, preds)
    sns.regplot(y_test, preds, label='prediction')
    plt.legend()
    plt.suptitle(f"{drugname} {save_name} : RMSE = {error}")
    
create_svm(merged, 'ABT-737', apop_genes, 'apoptosis_subset')

In [ ]:
merged.head(5)

In [ ]:
sort_ccle_gn = list(sorted(good_std))


XGBoost parameters


https://xgboost.readthedocs.io/en/latest/parameter.html

In [ ]:
# fit model no training data
params = dict(
    # general params
    booster='gbtree',
    gpu_id=0,
    seed=100,
    # regularization
    reg_alpha=1, 
    reg_lambda=5,
    num_parallel_tree = 4, 
#     num_boost_round = 16,
    tree_method='gpu_hist',
    max_bin=256,
    objective='reg:squarederror',
    eval_metric='rmse',
    learning_rate=0.05,
    max_depth=5, 
    min_child_weight=1,
#     gamma=0,
    subsample=.5, # use half of data to resample
#     colsample_bytree=.8,
)

def create_importance_model(df, drugname, subset_list, save_name):
    subset_genes = list(sorted(set(subset_list).intersection(set(df.columns.values))))
    X = df[subset_genes]
    y = df[drugname]
    X = preprocessing.StandardScaler().fit(X).transform(X)
    poly = PolynomialFeatures(degree=2)
    X = poly.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2, 
        shuffle=True, 
        random_state=69,
    )

    
    # organize data into xgb data matrix
    all_data = xgb.DMatrix(data=X, label=y)
    train = xgb.DMatrix(data=X_train, label=y_train)
    test = xgb.DMatrix(data=X_test, label=y_test)

    # add gene names as feature labels
#     all_data.feature_names = subset_genes
#     train.feature_names = subset_genes
#     test.feature_names = subset_genes
    
    num_round = 100000
    results = dict()
    model = xgb.train(
        params, train, num_round,
        verbose_eval=False,
        early_stopping_rounds=100,
        evals=[(train, 'train'), (test, 'valid')],
        evals_result=results,
        
    )  
    
    feature_scores = model.get_fscore()
    s = pd.Series(list(feature_scores.values()), index=feature_scores)
    print(s.sort_values(ascending=False).head(5))
    # retrieve performance metrics
    epochs = len(results['train']['rmse'])
    x_axis = range(0, epochs)
    # plot log loss
    plt.figure(figsize=(12, 6))
    plt.subplot(121)
    plt.plot(x_axis, results['train']['rmse'], label='Train')
    plt.plot(x_axis, results['valid']['rmse'], label='Test')
    plt.legend()
    
    #trained 
    t_preds = model.predict(train, ntree_limit=model.best_ntree_limit)

    #predictions
    preds = model.predict(test)
    error = mean_squared_error(y_test, preds)
    error = "{0:.3f}".format(np.sqrt(error))
    print(f"MSE: {error}")

    # simple plot
    
    plt.subplot(122)
    sns.regplot(y_train, t_preds, label='training')
    sns.regplot(y_test, preds, label='prediction')
    plt.legend()
    plt.suptitle(f"{drugname} {save_name} : RMSE = {error}")

    plt.savefig(f'pred_{save_name}.png')
    

In [ ]:
create_importance_model(merged, 'ABT-737', sort_ccle_gn, 'apoptosis_subset')

In [ ]:
drugs = sorted([i for i in drugs if isinstance(i, str)])

In [ ]:
errors = []
for i in drugs[:5]:
    errors.append(create_importance_model(merged, 'ABT-737', apop_genes, 'apoptosis_subset'))

In [ ]:
cc_genes = reactome_df.loc[reactome_df.term=='cell cycle homo sapiens r-hsa-1640170']['gene_list'].values
cc_genes = list(cc_genes[0])
create_importance_model(merged, 'ABT-737', cc_genes, 'cc_subset')

In [ ]:
big_terms = reactome_df.loc[(reactome_df.n_genes<1000) & (reactome_df.n_genes>500)].term.values
for i in big_terms:
    savename = i.replace(' ', '_').split('_homo')[0]
    genes = reactome_df.loc[reactome_df.term==i]['gene_list'].values
    create_importance_model(merged, drugs[0], set(list(genes)[0]), savename)

In [ ]:

def run_parameter_scan(df, drugname):
    X = df[sort_ccle_gn]
    y = df[drugname]

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2, 
        shuffle=False, 
        random_state=123,
    )

    # organize data into xgb data matrix
    train = xgb.DMatrix(data=X_train, label=y_train)
    test = xgb.DMatrix(data=X_test, label=y_test)

    # add gene names as feature labels
    train.feature_names = sort_ccle_gn
    test.feature_names = sort_ccle_gn
    num_round = 1000

    param_tuning = {
        'learning_rate': [0.1],
        'max_depth': [3, 5, 7, 10],
#         'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'objective': ['reg:squarederror'],
        'tree_method' : ['gpu_hist'],
        'eval_metric' :['rmse'],
        'reg_alpha':[1], 
        'reg_lambda':[30], 
        'num_parallel_tree':[4], 
    
    }

    xgb_model = XGBRegressor(
        n_estimators=num_round, 
        n_jobs=12,
        booster='gbtree'
    )

    gsearch = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_tuning,               
        cv=2,
        n_jobs=1,
        verbose=0
    )

    gsearch.fit(
        X_train, y_train, 
        early_stopping_rounds=50, 
#         eval_metric="mae",
        eval_set=[[X_train, y_train], [X_test, y_test]],
        verbose=False
    )
    print(gsearch.best_params_)
    
        #trained 
    t_preds = gsearch.predict(X_train)
    #predictions
    preds = gsearch.predict(X_test)
    error = mean_squared_error(y_test, preds)
    error = "{0:.3f}".format(np.sqrt(error))
    
    print(f"MSE: {error}")

    # simple plot
    plt.figure()
    plt.subplot(111)
    sns.regplot(y_train, t_preds, label='training')
    sns.regplot(y_test, preds, label='prediction')
    plt.legend()
    plt.suptitle(f"{drugname} : RMSE = {error}")

    plt.savefig(f'pred_{drugname}.png')
    return gsearch.best_params_
# run_parameter_scan(merged, drugs[0])

In [ ]:
best_params = dict()
for i in drugs[0:10]:
    best_params[i] = run_parameter_scan(merged, i)
    

# NN with PyTorch

In [ ]:
class Net(nn.Module):
    def __init__(self, D_in, H1, D_out):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(D_in, H1)
        self.linear2 = nn.Linear(H1, D_out)
        self.dropout = nn.Dropout(p=0.5)
        self.Tanh = nn.Tanh()
    def forward(self, x):
        y_pred = self.dropout(self.linear1(x))
        y_pred = self.dropout(self.Tanh(y_pred))
        y_pred = self.dropout(self.linear2(y_pred))
        y_pred = self.dropout(self.Tanh(y_pred))
        return y_pred
    

In [ ]:
import time
from copy import deepcopy
def run_nn(df, gene_list, save_name, drugname, verbose=False):
    torch.cuda.empty_cache()
    dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     print(drugname)
#     dev = 'cpu'
#     print('Using device:', dev)
    ib = list(set(gene_list).intersection(set(df.columns.values)))
    # ib = list(set(good_std).intersection(numeric_data.columns.values))
    X = df[ib].values
    y = df[drugs].values#.reshape(-1,1)
    X = preprocessing.StandardScaler().fit(X).transform(X)
    y = preprocessing.StandardScaler().fit(y).transform(y)
#     poly = PolynomialFeatures(degree=2)
#     X = poly.fit_transform(X)
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X,
            y,
            test_size=0.5, 
            shuffle=True, 
            random_state=69,
        )
    except:
        return
    
    dtype = torch.float32
    X_train = torch.tensor(X_train, dtype=dtype,  device=dev)
    X_test = torch.tensor(X_test, dtype=dtype,  device=dev)

    y_train = torch.tensor(y_train, dtype=dtype,  device=dev)
    y_test = torch.tensor(y_test, dtype=dtype,  device=dev)

#     print(X_train.shape, y_train.shape)
    D_in, D_out = X_train.shape[1], y_train.shape[1]
    H1 = int((D_in+D_out)/2)
#     H1 = 100
    model = Net(D_in, H1, D_out)
    model.to(dev)
    model.float()
    criterion = nn.MSELoss(reduction='mean')
    
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
    
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=1e-4,
        betas=(0.9, 0.999), 
        eps=1e-08, 
        weight_decay=0,
        amsgrad=True
    )
#     optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#     optimizer = torch.optim.ASGD(
#         model.parameters(), lr=0.01, lambd=0.001, 
#         alpha=0.0, t0=1000.0, weight_decay=0.
#     )

    n_epoch = 10000
    eval_per_epoch = n_epoch//10
    prev_loss = np.inf
    steps_wo_impro = 0
    best_state = deepcopy(model.state_dict())
    best_epoch = 0
    st = time.time()
    train, valid = [], []
    for epoch in range(n_epoch):
        if steps_wo_impro > eval_per_epoch:
#             print(f"Early stopping at {epoch}")
            break
        model.train()
        
        # use original method
        optimizer.zero_grad()
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
        if torch.isnan(loss):
            break        
        loss.backward()
        optimizer.step()
        with torch.no_grad():
        
            test_loss = criterion(model(X_test), y_test).item()

            train.append(loss.item())
            valid.append(test_loss)

            if test_loss < prev_loss:
                prev_loss = deepcopy(test_loss)
                steps_wo_impro = 0
                best_state = deepcopy(model.state_dict())
                best_epoch = epoch
            else:
                steps_wo_impro += 1
    #         if epoch < 1000:
    #             steps_wo_impro = 0
    #             prev_loss = np.inf
            if verbose:
                if (epoch+1) % eval_per_epoch == 0:
                    print(
                        'Epoch [{}/{}], Loss train: {:2.4e}, test {:2.4e} \
                        '.format(epoch+1, n_epoch, loss.item(), test_loss)
                    )
#     print(f'time : {time.time()-st}')
    del model
    
    test_model = Net(D_in, H1, D_out)
    test_model.load_state_dict(best_state)
    test_model.float()
    test_model.to(dev)
    test_model.eval()
    
    train_pred = test_model(X_train).cpu().detach().numpy()
    train_actual = y_train.cpu().detach().numpy()
    
    error = mean_squared_error(train_pred, train_actual)
    error = "{0:.3f}".format(np.sqrt(error))
#     print(f"Best epoch = {best_epoch} with error = {prev_loss} {np.sqrt(prev_loss)}")
#     print(f"\t {error}")  
    
    test_pred = test_model(X_test).cpu().detach().numpy()
    test_actual =  y_test.cpu().numpy()
    error = mean_squared_error(test_pred, test_actual)
#     return np.sqrt(error)
    error = "{0:.3f}".format(np.sqrt(error))
    
    print(f"\t {error}")  
    
    # simple plot
    plt.figure(figsize=(12, 6))
    plt.subplot(121)
    x_axis = range(0, len(train))
    plt.plot(best_epoch, prev_loss, marker='^')
    plt.plot(x_axis, train, label='Train')
    plt.plot(x_axis, valid, label='Test')
    plt.legend()

    plt.subplot(122)
    sns.regplot(train_pred[:,0], train_actual[:,0], label='Training')
    sns.regplot(test_pred[:,0], test_actual[:,0], label='Prediction')
    plt.legend()
    plt.suptitle(f"{drugname} {save_name} : RMSE = {error}")
#     plt.close()
    return np.sqrt(mean_squared_error(test_pred, test_actual))

In [ ]:
run_nn(merged, set(merged.columns.values), 'ABT-737', 'ABT-737', True)    

In [ ]:
from tqdm import tqdm

In [ ]:
errors = []
big_terms = list(reactome_df.loc[(reactome_df.n_genes<200) & (reactome_df.n_genes>100)].term.values)
# big_terms = list(reactome_df.loc[reactome_df.n_genes>100].term.values)
n_drugs = len(drugs[:])
print(len(big_terms[:]))
print(n_drugs)
for i in tqdm(drugs[:200], desc = 'Drugs'):
#     for j in tqdm(big_terms[:20], desc = 'Terms'):
    for j in big_terms[:]:
#         continue
        genes = list(reactome_df.loc[reactome_df.term==j]['gene_list'].values)[0]
        errors.append([i, j, run_nn(merged, genes, i,i)])
        

In [ ]:
from copy import deepcopy

In [ ]:
messed_up_errors = deepcopy(errors)
count = 0
for i in drugs[:200]:
    for j in big_terms[:]:
        pname = j.split(' homo')[0]
        messed_up_errors[count].insert(1, pname)
        count+=1

In [ ]:
error_df = pd.DataFrame(messed_up_errors, columns=['drug', 'pathway', 'error'])
error_df.tail(5)

In [ ]:
# error_df = error_df.pivot(index='drug', columns='pathway', values='error')
error_df = pd.pivot_table(error_df, index='drug', columns='pathway', values='error')
error_df.dropna(inplace=True)
error_df

In [ ]:
sns.clustermap(error_df, figsize=(18, 22), cmap=sns.cubehelix_palette(as_cmap=True), vmax=1);
plt.savefig('drug_by_term.png', bbox_inches='tight', dpi=300)

In [ ]:
for i in tqdm(drugs[:200], desc = 'Drugs'):
#     for j in tqdm(big_terms[:20], desc = 'Terms'):
    for j in big_terms[:]:
#         continue
        genes = list(reactome_df.loc[reactome_df.term==j]['gene_list'].values)[0]
        errors.append([i, j.split(' homo')[0], run_nn(merged, genes, i,i)])

In [ ]:
error_df = pd.DataFrame(errors, columns=['drug', 'pathway', 'error'])
error_df = error_df.pivot(index='drug', columns='pathway', values='error')
error_df.dropna(inplace=True)
error_df

In [ ]:
sns.clustermap(error_df, figsize=(12, 12), cmap=sns.cubehelix_palette(as_cmap=True), vmax=1);
plt.savefig('drug_by_term.png', bbox_inches='tight', dpi=300)

In [ ]:
cc_genes = reactome_df.loc[reactome_df.term=='cell cycle homo sapiens r-hsa-1640170']['gene_list'].values
cc_genes = list(cc_genes[0])
for i in drugs[:2]:
    run_nn(merged, cc_genes, i, i, True)
#     run_nn(good_std, i, i, True)

In [ ]:
cc_genes = reactome_df.loc[reactome_df.term=='cell cycle homo sapiens r-hsa-1640170']['gene_list'].values
cc_genes = list(cc_genes[0])
for i in drugs:
#     run_nn(cc_genes, i, i, True)
    run_nn(good_std, i, i, True)

In [ ]:
cc_genes = reactome_df.loc[reactome_df.term=='cell cycle homo sapiens r-hsa-1640170']['gene_list'].values
cc_genes = list(cc_genes[0])
run_nn(cc_genes, 'cc', True)

In [ ]:
big_terms = list(reactome_df.loc[(reactome_df.n_genes<500) & (reactome_df.n_genes>100)].term.values)
for i in big_terms[:]:
    print(i)
    savename = i.replace(' ', '_').split('_homo')[0]
    genes = list(reactome_df.loc[reactome_df.term==i]['gene_list'].values)[0]
    run_nn(genes, 'savename')

In [ ]:
big_terms = list(reactome_df.loc[(reactome_df.n_genes<1000) & (reactome_df.n_genes>500)].term.values)
for i in big_terms[:]:
    print(i)
    savename = i.replace(' ', '_').split('_homo')[0]
    genes = list(reactome_df.loc[reactome_df.term==i]['gene_list'].values)[0]
    run_nn(genes, 'savename')

## Connectivety map data 
We will use fold change data of the CMAPs 12,328 genes
![title](https://clue.io/connectopedia/images/data_levels_images/image_0.png)


In [ ]:
cmap_cl = pd.read_csv('data/LINCS/cellinfo_beta.txt', delimiter='\t')
cmap_cl.head(5)

In [ ]:
cmap_ccle_cl = set(cmap_cl.dropna(subset=['ccle_name']).ccle_name.values)

overlap = treatment.reset_index()
overlap = overlap.loc[overlap.cell_line.isin(cmap_ccle_cl)]
overlap.head(5)

In [ ]:
plt.figure(figsize=(12,12))
plt.title(f'{overlap.shape[1]} drugs: {overlap.shape[0]} cell lines')
overlap.set_index('cell_line', inplace=True)
sns.heatmap(overlap)

In [ ]:
cmap_gn = pd.read_csv('data/LINCS/geneinfo_beta.txt', delimiter='\t')
cmap_gn.head(5)

In [ ]:
len(cmap_gn.gene_symbol.unique())
cmap_genes = set(cmap_gn.gene_symbol.unique())

## CCLE expression

In [ ]:
ccle_mirna = pd.read_csv('data/ccle/CCLE_RNAseq_genes_rpkm_20180929.gct',
                         delimiter='\t', skiprows=2)
print(ccle_mirna.shape)
ccle_mirna = ccle_mirna.loc[ccle_mirna.Description.isin(cmap_genes)]
print(ccle_mirna.shape)

In [ ]:
ccle_mirna.set_index('Description', inplace=True)
ccle_mirna.drop('Name', axis=1, inplace=True)
ccle_mirna.head(5)

In [ ]:
ccle_mirna.describe()

In [ ]:
ccle_mirna.columns

In [ ]:
print(sorted(i for i in ccle_mirna.columns if i.startswith('PC')))

In [ ]:
print(sorted(i for i in ccle_mirna.columns if i.startswith('H') and 'LUNG' in i))

In [ ]:
ccle_mirna.T.head(5)

In [ ]:
ccle_mirna.T.describe()

In [ ]:
cols = set(ccle_mirna.columns.values)
inboth = cols.intersection(cmap_ccle_cl)

print(ccle_mirna.shape)
ccle_mirna_filtered = ccle_mirna[inboth]
print(ccle_mirna_filtered.shape)
ccle_mirna_filtered = ccle_mirna_filtered[ccle_mirna_filtered.T.mean() > 1]
ccle_mirna_filtered = ccle_mirna_filtered[(ccle_mirna_filtered.T.std()/ccle_mirna_filtered.T.mean()) > 1]
print(ccle_mirna_filtered.shape)

In [ ]:
ccle_mirna_filtered.T.columns

In [ ]:
plt.figure(figsize=(12,12))
sns.boxplot(data=ccle_mirna_filtered.T[['PLCH2', 'NPPB']])

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(ccle_mirna_filtered, vmax=10, )

In [ ]:

sns.clustermap(ccle_mirna_filtered, vmax=10, figsize=(12,12))